In [1]:
import os
import sqlite3
from pathlib import Path
import pandas as pd
import shutil

#gather data from a specific csv file and return as a pandas df
BASE = Path.cwd()
DATA = BASE / "training_data"
#Make sure training_data folder exists
DATA.mkdir(parents= True, exist_ok= True)

In [ ]:
#Get the user path 
user_path = os.path.expanduser("~")
#Make the path to the kaggle json
downloads_path = os.path.join(user_path, 'Downloads/kaggle.json')

#Make the .kaggle folder in user
kaggle_folder = os.path.join(user_path, ".kaggle")
os.makedirs(kaggle_folder, exist_ok= True)

print(os.path.exists(downloads_path))

if os.path.exists(downloads_path):
    try:
        print("moving kaggle.json to user")
        #Make .kaggle folder path and check it exists
        
        #Move json to the folder
        shutil.move(downloads_path, kaggle_folder)
    except Exception as e:
        print(f"Error when trying to move kaggle.json: {e}")
else:
    #attempt to check if kaggle.json already exists
    final = os.path.join(kaggle_folder, "kaggle.json")
    if os.path.exists(final):
        print("kaggle json already exists in user")
    else:
        print("please download kaggle.json")

#print("Successfully moved json file")

In [ ]:
#TO USE THIS API you must have a .kaggle folder in your 'C:\NAME' directory -> then paste the kaggle.json authenticator
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

print(api.dataset_list_files('wordsforthewise/lending-club').files)

api.dataset_download_files('wordsforthewise/lending-club', path= DATA, unzip=True)

In [38]:
def retrieve_training_csv(): 
    """ Function that returns all the csv files in the training data folder as a dataframe object """ 
    csv_list = list(DATA.glob("**/*.csv")) 
    return_list = [] 
    print(csv_list)

    #traverse each item inthe data path, and if a file ending in .csv is found, turn it into a df and append to return list 
    for item in csv_list: 
        if os.path.isfile(item): 
            print(f"{item} is a file") 
            return_list.append(pd.read_csv(item)) 
        else: print(f"{item} is folder or dir")

    if return_list: 
        return return_list 
    else: 
        print(f"No csv files found in {DATA}") 
        return []

def get_dir_size(path): 
    """ Get directory size in MBs """ 
    total = 0 
    for dirpath, dirs, files in os.walk(path):
        for f in files:
            file_path = os.path.join(dirpath, f)
            
            try:
                total += os.path.getsize(file_path) 
                print(f"{total/1000000} MB")
            except: 
                continue 
        return total/1000000
    
def delete_large_files(): 
    """ Function that deletes the large files downloaded from kaggle to save space 
    Deletes files to prevent any storage errors when pushing code to github """

    for paths in DATA.glob("**/*"): 
        if os.path.isdir(paths): 
            try:
                size = get_dir_size(paths) 
                print(f"size of {paths} is {size}")

                if size > 100: 
                    print(f"deleting {paths}") 
                    shutil.rmtree(paths) 

            except Exception as e:
                print(f"Error when trying to get size or delete file: {e}")
        else:
            if paths.suffix == ".gz":
                os.remove(paths)
                #print(paths)
        

df_list = retrieve_training_csv()

[WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/accepted_2007_to_2018q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/rejected_2007_to_2018q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv')]
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\accepted_2007_to_2018q4.csv is folder or dir
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\rejected_2007_to_2018q4.csv is folder or dir
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\accepted_2007_to_2018q4.csv\accepted_2007_to_2018Q4.csv is a file


C:\Users\Saul\AppData\Local\Temp\ipykernel_4384\3710330960.py:11: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  return_list.append(pd.read_csv(item))


c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\rejected_2007_to_2018q4.csv\rejected_2007_to_2018Q4.csv is a file


In [47]:
#create different pointers to the csvs in the list for easier access
raw_accepted_loans = df_list[0].copy()
raw_rejected_loans = df_list[1].copy()
#@raw_loans_paid_info = df_list[2]

In [48]:
raw_rejected_loans.head(10)

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0
5,15000.0,2007-05-27,Trinfiniti,645.0,0%,105xx,NY,3 years,0.0
6,10000.0,2007-05-27,NOTIFYi Inc,693.0,10%,210xx,MD,< 1 year,0.0
7,3900.0,2007-05-27,For Justin.,700.0,10%,469xx,IN,2 years,0.0
8,3000.0,2007-05-28,title?,694.0,10%,808xx,CO,4 years,0.0
9,2500.0,2007-05-28,timgerst,573.0,11.76%,407xx,KY,4 years,0.0


In [ ]:
al_columns = list(raw_accepted_loans.columns)
print(al_columns)
raw_accepted_loans['loan_status'].head(10)
num_cols = [
    "id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "annual_inc",
    "dti", "delinq_2yrs", "fico_range_low", "fico_range_high", "inq_last_6mths",
    "open_acc", "revol_bal", "revol_util", "total_acc", "pub_rec_bankruptcies"
]

text_cols = [
    "home_ownership", "loan_status", "purpose", "application_type", "verification_status"
]


"""
raw_specific_al = raw_accepted_loans[target_columns]
raw_specific_al.head(10).T
"""

**Clean Accepted Loans Data**
- Provide a data summary
- Standardize all data in the columns to reasonable types (ex: object -> float)
- Clean up any bad data (missing values, duplicates, etc)


In [ ]:
#only find columns with valid ids (not string or empty)
cleaned_accepted_loans = raw_accepted_loans[num_cols + text_cols].copy()
cleaned_accepted_loans = cleaned_accepted_loans[pd.to_numeric(cleaned_accepted_loans['id'], errors='coerce').notna()]

#convert id column to int instead of obj
cleaned_accepted_loans['id'] = cleaned_accepted_loans['id'].astype('int64')

In [ ]:
#fix the the term column to make it float (and in months)
cleaned_accepted_loans.rename(columns = {'term': 'term_months'}, inplace = True)
num_cols[3] = 'term_months'
cleaned_accepted_loans['term_months'] = cleaned_accepted_loans['term_months'].str.replace(" months", "")
cleaned_accepted_loans['term_months'] = cleaned_accepted_loans['term_months'].astype('int64')

In [ ]:
cleaned_accepted_loans.isna().sum()

In [ ]:
#find any na rows and fill them as needed
for rows in num_cols:
    cleaned_accepted_loans.loc[:, rows] = cleaned_accepted_loans[rows].fillna(cleaned_accepted_loans[rows].median())

cleaned_accepted_loans.isna().sum()


In [ ]:
#Get a sample of the cleaned up 
sampled_accepted_loans = cleaned_accepted_loans.sample(200000, random_state = 821)
sampled_accepted_loans

**Clean Rejected Loan Data**

In [41]:
raw_rejected_loans.isna().sum()

Amount Requested               0
Application Date               0
Loan Title                  1305
Risk_Score              18497630
Debt-To-Income Ratio           0
Zip Code                     293
State                         22
Employment Length         951355
Policy Code                  918
dtype: int64

In [42]:
#Drop unneeded columns 
raw_rejected_loans = raw_rejected_loans.drop(columns=['Risk_Score', 'Zip Code', 'State', 'Policy Code', 'Employment Length'])
rl_cols = list(raw_rejected_loans.columns)
print(rl_cols)

raw_rejected_loans.dtypes

['Amount Requested', 'Application Date', 'Loan Title', 'Debt-To-Income Ratio']


Amount Requested        float64
Application Date         object
Loan Title               object
Debt-To-Income Ratio     object
dtype: object

In [43]:
raw_rejected_loans['Debt-To-Income Ratio'] = raw_rejected_loans['Debt-To-Income Ratio'].str.replace('%', '')
raw_rejected_loans['Debt-To-Income Ratio'].value_counts()

Debt-To-Income Ratio
100         1362556
-1          1203063
0           1045102
9999          76984
1.2           32659
             ...   
44805.67          1
2602.53           1
73243             1
833.25            1
28475             1
Name: count, Length: 126145, dtype: int64

In [44]:
#Fix any negative dti ratios
raw_rejected_loans['Debt-To-Income Ratio'] = raw_rejected_loans['Debt-To-Income Ratio'].astype('float64')
raw_rejected_loans.loc[raw_rejected_loans['Debt-To-Income Ratio'] < 0, 'Debt-To-Income Ratio'] = raw_rejected_loans['Debt-To-Income Ratio'].mean()
raw_rejected_loans['Debt-To-Income Ratio'].value_counts()

Debt-To-Income Ratio
100.000000      1362556
143.340051      1203063
0.000000        1045102
9999.000000       76984
1.200000          32659
                 ...   
44805.670000          1
2602.530000           1
73243.000000          1
833.250000            1
28475.000000          1
Name: count, Length: 126145, dtype: int64

In [49]:
#Standardize the Loan Title data
raw_rejected_loans['Loan Title'] = (
    raw_rejected_loans['Loan Title'].astype('string').str.lower()
    .str.replace(r'[^a-z\s]', ' ', regex = True)
    .str.replace(r'\s+', ' ', regex = True).str.strip()
)

In [53]:
raw_rejected_loans['Loan Title'].fillna(raw_rejected_loans['Loan Title'].value_counts().reset_index().iat[0,0])
raw_rejected_loans['Loan Title'].value_counts()

Loan Title
debt consolidation         12322066
other                       4698761
credit card refinancing     2298378
credit card                 1364140
home improvement            1190977
                             ...   
smmoore                           1
thad                              1
dougie                            1
freeup                            1
pay off debt and irs              1
Name: count, Length: 60730, dtype: Int64

In [55]:
def word_map(text):
    """
    Function to normalize the loan reason column in rejected dataframe
    """
    if pd.isna(text): text = '' 
    
    if any (word in text for word in ['debt', 'consol']):
        return 'debt_consolidation'
    if any (word in text for word in ['cc', 'credit card']):
        return 'credit_card'
    if any (word in text for word in ['construction', 'remodeling', 'drywall']):
        return 'home_improvement'
    if any (word in text for word in ['major purchase', 'big buy']):
        return 'major_purchase'
    if any (word in text for word in ['medical', 'hospital', 'health', 'medical bill']):
        return 'medical' 
    
    return 'other'

raw_rejected_loans['Loan Title'] = raw_rejected_loans['Loan Title'].apply(word_map)

raw_rejected_loans['Loan Title'].value_counts()

Loan Title
debt_consolidation    12350757
other                  9917873
credit_card            3672642
major_purchase          957587
medical                 749676
home_improvement           206
Name: count, dtype: int64

In [ ]:
#Fix any missing values in the rejection loans
for rows in rl_cols:
    #fix the rows that are of float type
    if cleaned_rejected_loans[rows].dtypes == 'float64':
        print(f"replacing {rows} with median float")
        cleaned_rejected_loans.loc[:, rows] = cleaned_rejected_loans[rows].fillna(cleaned_rejected_loans[rows].median())
    else:
        #get the highest repeated string and replace N/A's with that string
        print(f"replacing {rows} na's with most repeated string")
        replacement_string = cleaned_rejected_loans[rows].value_counts().reset_index().at[0,rows]
        cleaned_rejected_loans.loc[:, rows] = cleaned_rejected_loans[rows].fillna(replacement_string)

In [ ]:
cleaned_rejected_loans.isna().sum()
print(len(cleaned_rejected_loans))

cleaned_rejected_loans.head(10)

In [ ]:
#Get csv of accepted loans from 2023 HMDA
import requests
try:
    #response = requests.get("https://ffiec.cfpb.gov/v2/data-browser-api/view/nationwide/csv?years=2023&actions_taken=1&loan_purpose=1")
    accepted_response = requests.get("https://files.ffiec.cfpb.gov/data-browser/datasets/2023/filtered-queries/one-year/a5a77b5e9528ccb95aae0dc60cea9d70.csv", stream= True)
    accepted_response.raise_for_status()
except requests.HTTPError as e:
    print(f"Error getting HDMI data: {e}")

In [ ]:
#turning response into csv
import io
import csv
raw_string = accepted_response.text
split_string = raw_string.split('\n')
#write to a csv in the training data folder
string_columns = split_string[0].split(',')
string_data = [rows.split(',') for rows in split_string[1:]]

hdma_path = DATA / 'hdma_accepted_raw.csv'

with open(hdma_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(string_columns)
    csv_writer.writerows(string_data)

In [ ]:
import io
import csv
hdma_path = DATA / 'hdma_accepted_raw.csv'
space_saver = pd.read_csv(hdma_path, low_memory= False)

In [ ]:
hdma_accepted_parquet = DATA / 'hdma_accepted_raw.parquet.gzip'
#space_saver.to_parquet(hdma_accepted_parquet)
for columns in space_saver.columns:
    if space_saver[columns].dtype == 'object':
        print('object')
        space_saver[columns] = space_saver[columns].astype('string')
    else:
        print(space_saver[columns].dtype)

In [ ]:
#columns that do not affect model predictions in any meaningful way. dropped in order to save space when uploading to repository
drop_cols = [
    'tract_population', 'tract_minority_population_percent', 'ffiec_msa_md_median_family_income', 'tract_to_msa_income_percentage',
    'tract_owner_occupied_units', 'tract_one_to_four_family_homes', 'tract_median_age_of_housing_units', 'aus-1', 'aus-2', 'aus-3', 'aus-4',
    'aus-5', 'applicant_ethnicity-1', 'applicant_ethnicity-2', 'applicant_ethnicity-3', 'applicant_ethnicity-4', 'applicant_ethnicity-5',
    'co-applicant_ethnicity-1', 'co-applicant_ethnicity-2', 'co-applicant_ethnicity-3', 'co-applicant_ethnicity-4', 'co-applicant_ethnicity-5',
    'applicant_ethnicity_observed', 'co-applicant_ethnicity_observed', 'applicant_race-1', 'applicant_race-2', 'applicant_race-3', 'applicant_race-4',
    'applicant_race-5', 'co-applicant_race-1', 'co-applicant_race-2', 'co-applicant_race-3', 'co-applicant_race-4', 'co-applicant_race-5',
    'applicant_race_observed', 'co-applicant_race_observed', 'applicant_sex', 'co-applicant_sex', 'applicant_sex_observed', 'co-applicant_sex_observed',
    'applicant_age', 'co-applicant_age', 'applicant_age_above_62', 'co-applicant_age_above_62'
    ]

In [ ]:
space_saver.drop(columns = drop_cols).to_parquet(hdma_accepted_parquet, compression='gzip')

In [ ]:
recovered = pd.read_parquet(hdma_accepted_parquet, columns=temp_num_cols + temp_text_cols)

#print(temp_num_cols + temp_text_cols)

In [ ]:
recovered.head(10)

In [ ]:
remove_index = []

#get the column numbers needed
for columns in temp_num_cols:
    remove_index.append(string_columns.index(columns))

for columns in temp_text_cols:
    remove_index.append(string_columns.index(columns))

print(remove_index)

In [ ]:
finalized_cols = [row for row in string_columns if string_columns.index(row) in remove_index]
finalized_text = [row for row in string_data if string_data.index(row) in remove_index]
print((finalized_cols))
print(len(string_data))

In [ ]:
temp_list = [i for i in range(0,100) if i not in remove_index]
print(temp_list)
split_string.clear()
print(len(temp_list))

In [ ]:
print(string_columns)
print(string_data[0])
hmda_accepted = pd.DataFrame(string_data, columns=string_columns)


In [6]:
#https://ffiec.cfpb.gov/documentation/publications/loan-level-datasets/lar-data-fields#loan_amount
temp_num_cols = [
    'activity_year', 'action_taken', 'preapproval', 'loan_purpose', 'loan_amount', 'loan_to_value_ratio',
    'loan_term', 'income', 'debt_to_income_ratio'
    ]
temp_text_cols = [
    'derived_loan_product_type', 'applicant_credit_score_type', 'co-applicant_credit_score_type', 'denial_reason-1'
    ]

"""
num_cols = [
    "id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "annual_inc",
    "dti", "delinq_2yrs", "fico_range_low", "fico_range_high", "inq_last_6mths",
    "open_acc", "revol_bal", "revol_util", "total_acc", "pub_rec_bankruptcies"
]

text_cols = [
    "home_ownership", "loan_status", "purpose", "application_type", "verification_status"
]
"""

'\nnum_cols = [\n    "id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "annual_inc",\n    "dti", "delinq_2yrs", "fico_range_low", "fico_range_high", "inq_last_6mths",\n    "open_acc", "revol_bal", "revol_util", "total_acc", "pub_rec_bankruptcies"\n]\n\ntext_cols = [\n    "home_ownership", "loan_status", "purpose", "application_type", "verification_status"\n]\n'

In [ ]:
hmda_accepted.iloc[0:5,20:30]

In [ ]:
hmda_accepted = hmda_accepted[temp_num_cols + temp_text_cols]
hmda_accepted.columns

In [ ]:
hmda_accepted.head(10)

In [ ]:
#Get csv of rejected loans from 2023 HMDA
import requests
try:
    rejected_response = requests.get("https://ffiec.cfpb.gov/v2/data-browser-api/view/nationwide/csv?years=2023&actions_taken=3&loan_purpose=1")
    rejected_response.raise_for_status()
except requests.HTTPError as e:
    print(f"Error getting HDMI data: {e}")

In [ ]:
rejected_string = rejected_response.text
split_string = rejected_string.split('\n')
#write to a csv in the training data folder
rejected_columns = split_string[0].split(',')
rejected_data = [rows.split(',') for rows in split_string[1:]]

In [3]:
#print(len(rejected_columns))
#print(rejected_data[0])
hdma_rejected_path = DATA / 'hdma_rejected_raw.csv'

In [ ]:
#Write raw data to unique csv
import csv
with open(hdma_rejected_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(rejected_columns)
    csv_writer.writerows(rejected_data)

In [ ]:
#Temporarily read csv, drop unneeded columns, and parquet the file to save space
rejected_temp = pd.read_csv(hdma_rejected_path, low_memory= False)

hdma_rejected_parquet = DATA / 'hdma_rejected_raw.parquet.gzip'
#space_saver.to_parquet(hdma_accepted_parquet)
for columns in rejected_temp.columns:
    if rejected_temp[columns].dtype == 'object':
        print('object')
        rejected_temp[columns] = rejected_temp[columns].astype('string')

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Saul\\Desktop\\Credit Risk Assessment\\credit-risk-assessment\\ETL\\training_data\\hdma_rejected_raw.csv'

In [ ]:
rejected_temp.drop(columns = drop_cols).to_parquet(hdma_rejected_parquet, compression='gzip')

In [7]:
hdma_accepted_parquet = DATA / 'hdma_accepted_raw.parquet.gzip'
hdma_rejected_parquet = DATA / 'hdma_rejected_raw.parquet.gzip'
accepted_recovered = pd.read_parquet(hdma_accepted_parquet, columns=temp_num_cols + temp_text_cols)
rejected_recovered = pd.read_parquet(hdma_rejected_parquet, columns=temp_num_cols + temp_text_cols)

In [ ]:
accepted_recovered.isna().sum()
accepted_recovered.head(10)

In [8]:
rejected_recovered.isna().sum()
rejected_recovered.head(10)
#rejected_recovered.dtypes

,activity_year,action_taken,preapproval,loan_purpose,loan_amount,loan_to_value_ratio,loan_term,income,debt_to_income_ratio,derived_loan_product_type,applicant_credit_score_type,co-applicant_credit_score_type,denial_reason-1
0,2023.0,3.0,2.0,1.0,545000.0,80.00000,360,110.0,50%-60%,Conventional:First Lien,2.0,9.0,1.0
1,2023.0,3.0,2.0,1.0,315000.0,98.97400,360,65.0,50%-60%,FHA:First Lien,3.0,9.0,4.0
2,2023.0,3.0,2.0,1.0,255000.0,96.50000,360,39.0,>60%,FHA:First Lien,2.0,10.0,1.0
3,2023.0,3.0,2.0,1.0,425000.0,90.00000,360,85.0,>60%,Conventional:First Lien,2.0,9.0,1.0
4,2023.0,3.0,2.0,32.0,425000.0,78.87900,360,158.0,30%-<36%,FHA:First Lien,1.0,9.0,1.0
5,2023.0,3.0,2.0,32.0,215000.0,84.70600,360,92.0,>60%,VA:First Lien,3.0,10.0,1.0
6,2023.0,3.0,2.0,1.0,245000.0,96.50000,360,48.0,50%-60%,FHA:First Lien,1.0,10.0,1.0
7,2023.0,3.0,2.0,32.0,565000.0,76.08700,360,134.0,50%-60%,FHA:First Lien,1.0,9.0,3.0
8,2023.0,3.0,2.0,1.0,415000.0,75.00000,360,177.0,50%-60%,Conventional:First Lien,3.0,9.0,1.0
9,2023.0,3.0,2.0,32.0,365000.0,40.78200,360,47.0,>60%,Conventional:First Lien,2.0,10.0,1.0


In [10]:
test_name = 'denial_reason-1'
print(rejected_recovered[test_name].value_counts().reset_index())
#print(rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

int_values = [
    'activity_year', 'action_taken', 'preapproval', 'loan_purpose', 'loan_amount', 'loan_term', 'applicant_credit_score_type',
    'co-applicant_credit_score_type', 'denial_reason-1'
]

float_values = [
    'loan_to_value_ratio', 'income', 'debt_to_income_ratio'
]

string_values = [
    'derived_loan_product_type'
]

   denial_reason-1   count
0              1.0  690467
1              3.0  565385
2              4.0  263717
3              7.0  198298
4              9.0  156939
5              6.0   74591
6              5.0   38532
7           1111.0   25720
8              2.0   23046
9              8.0     399


In [ ]:
#Convert any ranges into the middle of that range, or leave it at that range if its too broad
test_name = 'debt_to_income_ratio'
ranges = rejected_recovered[test_name].value_counts().reset_index()

rejected_recovered[test_name] = rejected_recovered[test_name].replace(">60%", 60)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("50%-60%", 55)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("20%-<30%", (29+20)/2)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("30%-<36%", (35+30)/2)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("<20%", 20)

rejected_recovered[test_name].value_counts().reset_index()

#rejected_recovered[column] = rejected_recovered[column].astype('float64', errors="ignore")

rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name].value_counts().reset_index()

In [ ]:
test_name = 'loan_term'
rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name].value_counts().reset_index()
print(len(rejected_recovered))

In [ ]:
test_name = 'income'
rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name].value_counts()

In [ ]:
test_name = 'loan_to_value_ratio'
#fix any rows that have exempt

rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name]

In [11]:
test_name = 'denial_reason-1'

rejected_recovered[test_name] = rejected_recovered[test_name].replace(1111, rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

In [18]:
print(type(rejected_recovered[test_name].value_counts().get('Exempt', default=False)))

<class 'bool'>


In [ ]:
#clean up dataframe and turn it into proper typings
for column in int_values:
    print(f"changing column: {column}")
    rejected_recovered[column] = rejected_recovered[column].fillna(rejected_recovered[column].value_counts().reset_index().iat[0, 0])
    try:
        rejected_recovered[column] = rejected_recovered[column].astype('int32')
    except Exception as e:
        print(f"skipping int conversion: {column}") 
        print(f"reason: {e}")
        continue  
    

for column in float_values:
    print(f"converting float: {column}")
    rejected_recovered[column] = rejected_recovered[column].fillna(rejected_recovered[column].value_counts().reset_index().iat[0, 0])   
    try:
        rejected_recovered[column] = rejected_recovered[column].astype('float64')
    except Exception as e:
        print(f"skipping float conversion: {column}")
        print(f"reason: {e}")

for column in string_values:
    rejected_recovered[column] = rejected_recovered[column].fillna(rejected_recovered[column].value_counts().reset_index().iat[0, 0])

rejected_recovered.dtypes

In [ ]:

"""
for column in rejected_recovered.columns:
    if rejected_recovered[column].dtype == 'float64':
"""

rejected_recovered.isna().sum()

In [ ]:
#Remove outliers from rejected df based on loan amount
rejected_recovered

"""
int_values = [
    'activity_year', 'action_taken', 'preapproval', 'loan_purpose', 'loan_amount', 'loan_term', 'applicant_credit_score_type',
    'co-applicant_credit_score_type', 'denial_reason-1'
]

float_values = [
    'loan_to_value_ratio', 'income', 'debt_to_income_ratio'
]
"""

loan_std = rejected_recovered['loan_amount'].std()
loan_mean = rejected_recovered['loan_amount'].mean()
print(loan_std)
print(loan_mean)
temp_z_data = rejected_recovered.copy()
temp_z_data['z_loan'] = ((rejected_recovered['loan_amount'] - loan_mean) / loan_std)
threshold = 3

#remove based on threshold
no_outliers_rejected = temp_z_data[(temp_z_data['z_loan'].abs()) <= threshold].drop(columns=['z_loan'])

In [ ]:
no_outliers_rejected['loan_amount'].value_counts().reset_index()
print(len(no_outliers_rejected))

In [ ]:
#Clean up accepted hdma loan (repeat of the rejected clean up process)
#Convert any ranges into the middle of that range, or leave it at that range if its too broad
test_name = 'debt_to_income_ratio'
ranges = accepted_recovered[test_name].value_counts().reset_index()

accepted_recovered[test_name] = accepted_recovered[test_name].replace(">60%", 60)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("50%-60%", 55)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("20%-<30%", (29+20)/2)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("30%-<36%", (35+30)/2)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("<20%", 20)

accepted_recovered[test_name].value_counts().reset_index()

#accepted_recovered[column] = accepted_recovered[column].astype('float64', errors="ignore")

accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name].value_counts().reset_index()

test_name = 'loan_term'
accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name].value_counts().reset_index()
print(len(accepted_recovered))

test_name = 'income'
accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name].value_counts()

test_name = 'loan_to_value_ratio'
#fix any rows that have exempt

accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name]

#clean up dataframe and turn it into proper typings
for column in int_values:
    print(f"changing column: {column}")
    accepted_recovered[column] = accepted_recovered[column].fillna(accepted_recovered[column].value_counts().reset_index().iat[0, 0])
    try:
        accepted_recovered[column] = accepted_recovered[column].astype('int32')
    except Exception as e:
        print(f"skipping int conversion: {column}") 
        print(f"reason: {e}")
        continue  
    

for column in float_values:
    print(f"converting float: {column}")
    accepted_recovered[column] = accepted_recovered[column].fillna(accepted_recovered[column].value_counts().reset_index().iat[0, 0])   
    try:
        accepted_recovered[column] = accepted_recovered[column].astype('float64')
    except Exception as e:
        print(f"skipping float conversion: {column}")
        print(f"reason: {e}")

for column in string_values:
    accepted_recovered[column] = accepted_recovered[column].fillna(accepted_recovered[column].value_counts().reset_index().iat[0, 0])

loan_std = accepted_recovered['loan_amount'].std()
loan_mean = accepted_recovered['loan_amount'].mean()
print(loan_std)
print(loan_mean)
temp_z_data = accepted_recovered.copy()
temp_z_data['z_loan'] = ((accepted_recovered['loan_amount'] - loan_mean) / loan_std)
threshold = 3

#remove based on threshold
no_outliers_accepted = temp_z_data[(temp_z_data['z_loan'].abs()) <= threshold].drop(columns=['z_loan'])

In [ ]:
#delete_large_files()

no_outliers_accepted.head(10)
no_outliers_accepted.isna().sum()